In [6]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [7]:
train_datagen = ImageDataGenerator(rescale=1/255)

In [13]:
train_generator = train_datagen.flow_from_directory(
        'images/training/',  
        target_size=(100, 100),
        class_mode='categorical',
)

Found 4000 images belonging to 5 classes.


In [9]:
test_datagen = ImageDataGenerator(rescale=1/255)

In [14]:
test_generator = test_datagen.flow_from_directory(
        'images/testing/',  
        target_size=(100, 100),
        class_mode='categorical',
)

Found 500 images belonging to 5 classes.


In [17]:
model = keras.Sequential([
    keras.layers.Conv2D(32,(3,3),input_shape=(100,100,3),activation='relu'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(16,(3,3),activation='relu'),
    keras.layers.MaxPooling2D(2,2),
    
    keras.layers.Flatten(),
    keras.layers.Dense(128,activation='relu'),
    keras.layers.Dense(32,activation='relu'),
    keras.layers.Dense(5,activation='softmax'),
])
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 98, 98, 32)        896       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 47, 47, 16)        4624      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 23, 23, 16)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 8464)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               1083520   
_________________________________________________________________
dense_10 (Dense)             (None, 32)                4128      
__________

In [137]:
# class myCallBack(keras.callbacks.Callback):
#     def on_epoch_end(self,epoch,logs={}):
#         n=input("\nStop Y/N\n")
#         if(n=='Y'):
#             print("\nManually Stopped")
#             self.model.stop_training = True
            
# callback=myCallBack()

In [19]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=8,  
    epochs=10,
    verbose=1,
#     callbacks=[callback],
    validation_data=test_generator,
    validation_steps=8,
)

model.save_weights('weights_cateogorical.h5')

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
125/125 [==============================] - 110s 880ms/step - loss: 0.2883 - acc: 0.9053 - val_loss: 0.1194 - val_acc: 0.9620
Epoch 2/10
125/125 [==============================] - 86s 691ms/step - loss: 0.0397 - acc: 0.9887 - val_loss: 0.0886 - val_acc: 0.9680
Epoch 3/10
 14/125 [==>...........................] - ETA: 1:11 - loss: 0.0138 - acc: 0.9955

KeyboardInterrupt: 

In [144]:
model_json = model.to_json()

In [145]:
with open("model.json",'w') as json_file:
    json_file.write(model_json)
json_file.close()

In [20]:
from tensorflow.keras.preprocessing import image as keras_Prep_image

In [21]:
def predict(path):
    img = keras_Prep_image.load_img(str(path),target_size=(100,100))
    img_arr = keras_Prep_image.img_to_array(img)
    img_arr = np.expand_dims(img_arr,axis=0)

    x=model.predict(img_arr)
    return x

In [89]:
def predict_from_array(array):
    img_arr = np.expand_dims(array,axis=0)
    x=model.predict(img_arr)
    return x

In [22]:
np.argmax(predict("images/training/gesture3/img190.jpg"))

2

In [24]:
predict("images/training/gesture3/img190.jpg")

array([[0., 0., 1., 0., 0.]], dtype=float32)

In [109]:
print(np.argmax(predict_from_array(cv2.merge((t,t,t)))),end='\r')

In [25]:
#----------------------------------------------Read Trained model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()


In [27]:
loaded_model = keras.models.model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("weights.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

Loaded model from disk


In [28]:
def predict_from_loaded_model(path):
    img = keras_Prep_image.load_img(str(path),target_size=(100,100))
    img_arr = keras_Prep_image.img_to_array(img)
    img_arr = np.expand_dims(img_arr,axis=0)

    x=loaded_model.predict(img_arr)
    return x

In [29]:
predict_from_loaded_model("images/training/gesture3/img190.jpg")

array([[0., 0., 1., 0., 0.]], dtype=float32)

In [30]:
# organize imports
import cv2
import imutils
import numpy as np

# global variables
bg = None

#-------------------------------------------------------------------------------
# Function - To find the running average over the background
#-------------------------------------------------------------------------------
def run_avg(image, aWeight):
    global bg
    # initialize the background
    if bg is None:
        bg = image.copy().astype("float")
        return

    # compute weighted average, accumulate it and update the background
    cv2.accumulateWeighted(image, bg, aWeight)

#-------------------------------------------------------------------------------
# Function - To segment the region of hand in the image
#-------------------------------------------------------------------------------
def segment(image, threshold=15):
    global bg
    # find the absolute difference between background and current frame
    diff = cv2.absdiff(bg.astype("uint8"), image)

    # threshold the diff image so that we get the foreground
    thresholded = cv2.threshold(diff,
                                threshold,
                                255,
                                cv2.THRESH_BINARY)[1]

    # get the contours in the thresholded image
    (cnts, _) = cv2.findContours(thresholded.copy(),
                                    cv2.RETR_EXTERNAL,
                                    cv2.CHAIN_APPROX_SIMPLE)

    # return None, if no contours detected
    if len(cnts) == 0:
        return
    else:
        # based on contour area, get the maximum contour which is the hand
        segmented = max(cnts, key=cv2.contourArea)
        return (thresholded, segmented)

#-------------------------------------------------------------------------------
# Main function
#-------------------------------------------------------------------------------
count=0
if __name__ == "__main__":
    # initialize weight for running average
    aWeight = 0.5

    # get the reference to the webcam
    camera = cv2.VideoCapture(0)

    # region of interest (ROI) coordinates
    top, right, bottom, left = 100, 300, 400, 600

    # initialize num of frames
    num_frames = 0

    # keep looping, until interrupted
    while(True):
        # get the current frame
        (grabbed, frame) = camera.read()

        # resize the frame
        frame = imutils.resize(frame, width=700)

        # flip the frame so that it is not the mirror view
        frame = cv2.flip(frame, 1)

        # clone the frame
        clone = frame.copy()

        # get the height and width of the frame
        (height, width) = frame.shape[:2]

        # get the ROI
        roi = frame[top:bottom, right:left]

        # convert the roi to grayscale and blur it
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7, 7), 0)

        # to get the background, keep looking till a threshold is reached
        # so that our running average model gets calibrated
        if num_frames < 30:
            run_avg(gray, aWeight)
        else:
            # segment the hand region
            hand = segment(gray)

            # check whether hand region is segmented
            if hand is not None:
                # if yes, unpack the thresholded image and
                # segmented region
                (thresholded, segmented) = hand

                # draw the segmented region and display the frame
                cv2.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))
                cv2.imshow("Thesholded", thresholded)

        # draw the segmented hand
        cv2.rectangle(clone, (left, top), (right, bottom), (0,255,0), 2)

        # increment the number of frames
        num_frames += 1

        # display the frame with segmented hand
        cv2.imshow("Video Feed", clone)

        # observe the keypress by the user
        keypress = cv2.waitKey(1) & 0xFF

        # if the user pressed "q", then stop looping
        if keypress == ord("q"):
            break
            
        elif(keypress == ord('s')):
            cv2.imwrite("images/testing/gesture1/img"+str(count)+".jpg",thresholded)
            count+=1
            print(count,end='\r')
        elif(keypress == ord('r')):
            num_frames=0
            bg=None
        
#         t=cv2.resize(thresholded,(100,100))
#         print(1+np.argmax(predict_from_array(cv2.merge((t,t,t)))),end='\r')
        cv2.imwrite("images/img_to_predict.jpg",thresholded)
        print(1+np.argmax(predict("images/img_to_predict.jpg")),end='\r')
# free up memory
camera.release()
cv2.destroyAllWindows()

NameError: name 'thresholded' is not defined

In [ ]:
camera.release()
cv2.destroyAllWindows()

ValueError: Error when checking input: expected conv2d_14_input to have 4 dimensions, but got array with shape (1, 100, 100)